In [19]:
from pycaret.classification import *
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

import sys
sys.path.insert(1, '../utils')
from run_models.train_predict import train_model, calification_model
from outliers.outlier import outlier

In [20]:
experiment_name = 'experiment_5'

## Lectura de Datos

In [21]:
data  = pd.read_csv(r"..\data\{}\pmod_base_model.csv".format(experiment_name))
data.head()

,nit_enmascarado,num_oblig_enmascarado,num_oblig_orig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,marca_pago,ajustes_banco,lote,avg_sld_cap_final_1,avg_sld_cap_final_stddev_ult3,avg_sld_cap_final_stddev_ult6,avg_nueva_altura_mora_1,avg_nueva_altura_mora_stddev_ult3,min_vlr_obligacion_stddev_ult3,avg_vlr_vencido_1,valor_cuota_mes_1,valor_cuota_mes_stddev_ult3,valor_cuota_mes_stddev_ult6,pago_total_1,pago_total_avg_ult3,porc_pago_1,porc_pago_avg_ult3,total_ing_1,total_ing_avg_ult3,total_ing_stddev_ult3,tot_activos_1,tot_activos_stddev_ult3,tot_activos_stddev_ult6,egresos_mes_1,egresos_mes_stddev_ult3,tot_patrimonio_stddev_ult3,tot_patrimonio_stddev_ult6,prob_propension_1,prob_propension_stddev_ult3,prob_propension_stddev_ult6,prob_propension_max_ult6,prob_alrt_temprana_1,prob_alrt_temprana_stddev_ult3,prob_alrt_temprana_stddev_ult6,prob_auto_cura_1,prob_auto_cura_stddev_ult3,prob_auto_cura_stddev_ult6
0,144677,944034,144069,202311,0.0,NO_PAGO,NO,2,3.951733e+06,2.688497e-10,3.292723e-10,73.967742,24.841908,96947.942534,610988.290323,1160581.0,315448.164749,353445.020129,0.0,0.000000,0.0,0.000000,5533000.0,6.523000e+06,700035.713375,56705000.0,1.317576e+06,2.576177e+07,722489.0,0.0,1.317576e+06,2.576177e+07,0.150296,0.059518,0.070604,0.291182,0.610527,0.005498,0.006131,0.107347,0.022619,0.053240
1,193430,470009,610333,202310,0.0,NO_PAGO,NO,2,1.966792e+06,0.000000e+00,0.000000e+00,64.000000,24.064492,46825.970418,181750.300000,212381.0,57832.455317,57802.501631,0.0,0.000000,0.0,0.000000,3400000.0,3.400000e+06,0.000000,51300000.0,0.000000e+00,2.565000e+07,1000000.0,0.0,0.000000e+00,1.815000e+07,0.317733,0.113442,0.099240,0.570639,0.735915,0.109572,0.087123,0.154782,0.076645,0.062913
2,61793,580684,499658,202401,NaN,FACTURACION_MES_SGTE,NO,2,1.809671e+06,1.615452e+04,3.753123e+04,1.000000,0.471405,15377.848376,10859.225806,48787.0,151.792256,161.978051,8172.0,133037.333333,17.0,272.000000,10865904.0,1.086590e+07,0.000000,70000000.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.925839,0.003060,0.002408,0.925839,0.378441,0.091614,0.085733,0.622551,0.013563,0.056557
3,198870,880527,204728,202310,0.0,IGUAL,NO,1,6.903930e+06,8.916558e+04,1.069897e+05,6.100000,5.364376,296681.795027,257228.966667,900672.0,223204.180619,223850.442124,901000.0,474327.000000,100.0,51.333333,2882000.0,2.882000e+06,0.000000,49076000.0,0.000000e+00,2.453800e+07,400000.0,0.0,0.000000e+00,2.403800e+07,0.683860,0.034618,0.055728,0.842226,0.609034,0.074364,0.155149,0.193738,0.086574,0.271958
4,34986,54648,1025694,202311,0.0,NO_PAGO,NO,2,3.128086e+06,2.688497e-10,1.474594e+06,52.483871,21.296844,68942.182622,184937.870968,249236.0,67877.405707,88652.936332,0.0,0.000000,0.0,0.000000,983166.0,9.221107e+05,86345.280455,13702225.0,0.000000e+00,6.459291e+06,800000.0,0.0,0.000000e+00,6.459291e+06,0.223442,0.219012,0.322461,0.755401,0.667432,0.193993,0.268648,0.240964,0.121755,0.203829


In [22]:
print("Cantidad de valores:", data.shape)
print("Cantidad de datos con valores infinitos: ", (data.isin([np.inf, -np.inf]).sum()).sum())
print("Cantidad de datos con valores nulos: ", (data.isna().sum()).sum())
print("Cantidad de datos con valores negativos: ", ((data.drop(columns=["marca_pago", "ajustes_banco"]) < 0).sum()).sum())

Cantidad de valores: (680800, 42)
Cantidad de datos con valores infinitos:  0
Cantidad de datos con valores nulos:  112549
Cantidad de datos con valores negativos:  28855


In [23]:
data_model = data[data["fecha_var_rpta_alt"] < 202401]
data_oot = data[data["fecha_var_rpta_alt"] == 202401]

## Encoding variables categoricas

In [24]:
print(data_model["marca_pago"].value_counts(normalize=True).to_dict())
encoding_dict = data_model["marca_pago"].value_counts(normalize=True).to_dict()
data_model["marca_pago"] = data_model["marca_pago"].map(encoding_dict)
data_model["marca_pago"].value_counts()

{'PAGO_MENOS': 0.3811572702907694, 'NO_PAGO': 0.33819386151542186, 'PAGO_MAS': 0.12223823627235147, 'IGUAL': 0.0735115292362002, 'FACTURACION_MES_SGTE': 0.07054629028369505, 'AJUSTES_BANCO': 0.007222160629721725, 'CANCELADO': 0.005399022615006397, 'NO_DATA': 0.001731629156833864}


marca_pago
0.381157    216593
0.338194    192179
0.122238     69462
0.073512     41773
0.070546     40088
0.007222      4104
0.005399      3068
0.001732       984
Name: count, dtype: int64

In [25]:
print(data_model["ajustes_banco"].value_counts(normalize=True).to_dict())
encoding_dict = data_model["ajustes_banco"].value_counts(normalize=True).to_dict()
data_model["ajustes_banco"] = data_model["ajustes_banco"].map(encoding_dict)
data_model["ajustes_banco"].value_counts()

{'NO': 0.9889626239109126, 'REDIFERIDOS': 0.009124489002219089, 'NO_DATA': 0.001731629156833864, 'AJUSTES': 0.00018125793003443902}


ajustes_banco
0.988963    561979
0.009124      5185
0.001732       984
0.000181       103
Name: count, dtype: int64

In [26]:
print(data_model["lote"].value_counts(normalize=True).to_dict())
encoding_dict = data_model["lote"].value_counts(normalize=True).to_dict()
data_model["lote"] = data_model["lote"].map(encoding_dict)
data_model["lote"].value_counts()

{1: 0.555395415054263, 2: 0.32764746564458314, 3: 0.11363464384576534, -99: 0.003322475455388552}


lote
0.555395    315604
0.327647    186186
0.113635     64573
0.003322      1888
Name: count, dtype: int64

## Escalamiento de variables

In [28]:
MM = MinMaxScaler()
data_model_norm = MM.fit_transform(data_model.drop(columns=['nit_enmascarado', 'num_oblig_enmascarado',
                                                            'num_oblig_orig_enmascarado', 'fecha_var_rpta_alt', 
                                                            'var_rpta_alt','marca_pago', 'ajustes_banco', 'lote']))
data_model_norm = pd.DataFrame(data_model_norm, columns=data_model.drop(columns=['nit_enmascarado', 'num_oblig_enmascarado',
                                                            'num_oblig_orig_enmascarado', 'fecha_var_rpta_alt', 
                                                            'var_rpta_alt','marca_pago', 'ajustes_banco', 'lote']).columns)
data_model_norm["var_rpta_alt"] = data_model["var_rpta_alt"].values
data_model_norm["marca_pago"] = data_model["marca_pago"].values
data_model_norm["lote"] = data_model["lote"].values
data_model_norm.head()

,avg_sld_cap_final_1,avg_sld_cap_final_stddev_ult3,avg_sld_cap_final_stddev_ult6,avg_nueva_altura_mora_1,avg_nueva_altura_mora_stddev_ult3,min_vlr_obligacion_stddev_ult3,avg_vlr_vencido_1,valor_cuota_mes_1,valor_cuota_mes_stddev_ult3,valor_cuota_mes_stddev_ult6,pago_total_1,pago_total_avg_ult3,porc_pago_1,porc_pago_avg_ult3,total_ing_1,total_ing_avg_ult3,total_ing_stddev_ult3,tot_activos_1,tot_activos_stddev_ult3,tot_activos_stddev_ult6,egresos_mes_1,egresos_mes_stddev_ult3,tot_patrimonio_stddev_ult3,tot_patrimonio_stddev_ult6,prob_propension_1,prob_propension_stddev_ult3,prob_propension_stddev_ult6,prob_propension_max_ult6,prob_alrt_temprana_1,prob_alrt_temprana_stddev_ult3,prob_alrt_temprana_stddev_ult6,prob_auto_cura_1,prob_auto_cura_stddev_ult3,prob_auto_cura_stddev_ult6,var_rpta_alt,marca_pago,lote
0,0.002545,5.703163e-19,6.984919e-19,0.046755,0.039898,0.000204,0.001461,0.002526,0.001876,0.001349,0.000000,0.000000,2.739924e-07,9.295633e-09,1.844331e-06,1.686983e-06,2.560345e-07,0.000006,4.537338e-07,0.000008,1.204148e-07,0.0,0.000001,0.000007,0.156728,0.142776,0.154516,0.303554,0.656524,0.012702,0.013471,0.113884,0.052014,0.115822,0.0,0.338194,0.327647
1,0.001267,0.000000e+00,0.000000e+00,0.040454,0.038649,0.000099,0.000435,0.000462,0.000344,0.000221,0.000000,0.000000,2.739924e-07,9.295633e-09,1.133332e-06,8.793102e-07,0.000000e+00,0.000006,0.000000e+00,0.000008,1.666667e-07,0.0,0.000000,0.000005,0.331331,0.272132,0.217183,0.594885,0.791359,0.253155,0.191425,0.164207,0.176252,0.136863,0.0,0.338194,0.327647
2,0.004446,1.891488e-04,2.269594e-04,0.003856,0.008616,0.000626,0.000615,0.001960,0.001327,0.000854,0.000867,0.000265,5.507524e-07,1.411559e-08,9.606654e-07,7.453447e-07,0.000000e+00,0.000005,0.000000e+00,0.000007,6.666667e-08,0.0,0.000000,0.000007,0.713127,0.083044,0.121958,0.878011,0.654919,0.171811,0.340890,0.205536,0.199085,0.591630,0.0,0.073512,0.555395
3,0.002014,5.703163e-19,3.128086e-03,0.033175,0.034204,0.000145,0.000442,0.000542,0.000404,0.000338,0.000000,0.000000,2.739924e-07,9.295633e-09,3.277216e-07,2.384769e-07,3.158034e-08,0.000002,0.000000e+00,0.000002,1.333333e-07,0.0,0.000000,0.000002,0.233004,0.525382,0.705696,0.787497,0.717716,0.448202,0.590269,0.255637,0.279986,0.443420,0.0,0.338194,0.327647
4,0.000621,1.138905e-06,3.041923e-05,0.002992,0.003584,0.000011,0.000035,0.000064,0.000000,0.000000,0.000000,0.000016,2.739924e-07,1.868516e-08,3.999995e-06,3.103448e-06,0.000000e+00,0.000020,9.172078e-06,0.000018,1.666667e-07,0.0,0.000027,0.000012,0.949959,0.011140,0.056996,0.960655,0.176154,0.102224,0.074490,0.878785,0.013534,0.011883,0.0,0.338194,0.113635


## Eliminacion de atípicos

In [29]:
# df_outliers = outlier(data_model_norm)
# data_model_norm = df_outliers.reset_index(drop=True)
# print(data_model_norm.shape)
# data_model_norm.head()

## Entrenar Modelo

In [ ]:

models = ['xgboost']
best_model = train_model(data_model_norm, "var_rpta_alt", models)

In [ ]:
best_model = best_model[0]
print(best_model)

## Tunear Modelo

In [ ]:
params = {
    'n_estimators' : [400, 450],
    'max_depth' : [9,10],
    'learning_rate' : [.1],
    'objective' : ['reg:squarederror'],
}

best_model_tuned = tune_model(best_model, fold=5, custom_grid=params, optimize='F1')
print(best_model_tuned)

In [ ]:
plot_model(best_model_tuned, plot='feature')
plot_model(best_model_tuned, plot = 'auc')
plot_model(best_model_tuned, plot = 'confusion_matrix')

## Calificar

In [ ]:
data_calification = data_oot
data_calification["marca_pago"] = data_calification["marca_pago"].map(encoding_dict)
data_calification.head()

In [ ]:
vars_model = data_model_norm.columns
vars_model[:-1]

In [ ]:
data_calification = calification_model(best_model_tuned, data_calification, vars_model[:-1], scaler = MM)
data_calification.head()

In [ ]:
data_calification["ID"] = data_calification["nit_enmascarado"].astype(str) + "#" + data_calification["num_oblig_orig_enmascarado"].astype(str) + "#" + data_calification["num_oblig_enmascarado"].astype(str)
new_dataframe = data_calification[["ID", "var_rpta_alt"]]
print(new_dataframe.isna().sum())
new_dataframe.to_csv(r"..\data\{}\pmod_base_model_calification.csv".format(experiment_name), index=False)